In [ ]:
from pathlib import Path
from scipy.signal import find_peaks
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mpl_toolkits.axes_grid1 import make_axes_locatable
import h5py
import copy
import pickle

# Project modules
from lib.Stokes import Stokes
from functions.plot_data import plot_data

In [ ]:
# Check that the Stokes object file exists
stokes_filename = "generated/objects/stokes.pickle"
my_file = Path(stokes_filename)
if not my_file.is_file():
    print("Stokes object does not exists (", stokes_filename,"), please run file init_data.py first")

# Load the objects from the files
print("Loading Stokes data from file", stokes_filename)
stokes_list = pickle.load(open(stokes_filename, "rb"))

In [ ]:
# Calculate first derivative
stokes_list['I'].calc_first_derivative()

In [ ]:
# Calculate second derivative
stokes_list['I'].calc_second_derivative()

In [ ]:
# Plot derived Intensity
wavelength = 6301.5
stokes_list['I'].plot_frame_d(25)
stokes_list['I'].plot_frame_dd(25)

In [ ]:
plt.plot(stokes_list['I'].data_n[300,300,:])

In [ ]:
plt.plot(stokes_list['I'].data_d[300,300,:])

In [ ]:
plt.plot(stokes_list['I'].data_dd[300,300,:])